In [1]:
import pandas as pd
import os
import json
from tqdm import tqdm

In [2]:
def load_data(path:str):
    csv_path = os.path.join("datasets", "titanic", "train.csv")
    return pd.read_csv(csv_path)

In [3]:
with open(r"./Project/2018/201800000") as f:
    data = json.load(f)

In [4]:
def extract_ref(ref:dict):
    info = ref["ref-info"]
    if "ref-title" in info.keys():
        if "ref-publicationyear" in info.keys():
            year = info["ref-publicationyear"]["@first"]
        else:
            year = "unknow"
        return year,info["ref-title"]["ref-titletext"]
    return (None,None)

In [5]:
list_ref = []
list_pub = []

for pub_year in tqdm(os.listdir(r"./Project")):
    for pub_name in tqdm(os.listdir(r"./Project/" + pub_year)):
        file_name = r"./Project/" + pub_year + "/"+ pub_name
        with open(file_name,encoding="utf8") as f:
            try:
                data = json.load(f)
            except:
                print(file_name)
                continue
            data = data["abstracts-retrieval-response"]["item"]["bibrecord"]
            abstracts = data["head"]["abstracts"]
            title = data["head"]["citation-title"]
            sourcetitle = data["head"]["source"]["sourcetitle"]
            list_pub.append((title,sourcetitle,abstracts))
            if data["tail"] is not None:
                list_referece = data["tail"]["bibliography"]["reference"]
                if isinstance(list_referece,dict):
                    year_ref,title_ref = extract_ref(list_referece)
                    if year_ref != None and title_ref != None:
                        list_ref.append((title,sourcetitle,year_ref,title_ref))
                else:
                    for i in list_referece:
                        year_ref,title_ref = extract_ref(i)
                        if year_ref != None and title_ref != None:
                            list_ref.append((title,sourcetitle,year_ref,title_ref))

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [03:03<00:00, 30.57s/it]


In [7]:
len(list_ref)

791710

In [13]:
df_ref = pd.DataFrame(list_ref,columns=["title","sourcetitle","year","title_ref"])
df_pub = pd.DataFrame(list_pub,columns=["title","sourcetitle","abstracts"])

# Autocomplete

In [145]:
df_pub = df_pub[~df_pub["abstracts"].isna()]
df_pub = df_pub[~df_pub["title"].isna()]
df_pub["len"] = df_pub["title"].str.len()
df_pub = df_pub.reset_index()

In [141]:
def auto_complete(query:str):
    df_contain = df_pub[df_pub["title"].str.lower().str.contains(query)]
    df_contain.loc[:,"ind"] = df_contain["title"].str.lower().str.find(query)
    return df_contain.sort_values(["ind","len"]).head(10)

In [147]:
result = auto_complete("machin")

C:\Users\win12\AppData\Local\Temp\ipykernel_14192\560698770.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_contain.loc[:,"ind"] = df_contain["title"].str.lower().str.find(query)


In [148]:
result.iloc[]

,index,title,sourcetitle,abstracts,len,ind
13499,13961,Machine Learning Based Design of Railway Prest...,Applied Sciences (Switzerland),© 2022 by the authors.The state-of-the-art des...,70,0
8770,9075,"Machine hermeneutics, postphenomenology, and f...",AI and Society,"© 2020, Springer-Verlag London Ltd., part of S...",74,0
18471,19093,Machine learning and statistical analysis for ...,Bioresource Technology,© 2022 Elsevier LtdTorrefaction is a remarkabl...,76,0
15725,16273,Machine Learning Techniques to Detect Failure ...,6th International Conference on Information Te...,© 2022 IEEE.This paper presents machine learni...,77,0
3459,3596,Machine learning to understand the immune-infl...,International Journal of Molecular Sciences,"© 2019 by the authors. Licensee MDPI, Basel, S...",79,0
14662,15166,Machine Reading Comprehension Using Multi-Pass...,ECTI Transactions on Computer and Information ...,"© 2022, ECTI Association. All rights reserved....",84,0
16408,16971,Machine-learning-based automated quantificatio...,PeerJ Computer Science,© Copyright 2022 Phanomchoeng et al.The plaque...,87,0
19200,19858,Machine Learning applications for Data Quality...,Journal of Physics: Conference Series,© Published under licence by IOP Publishing Lt...,91,0
21,23,Machine Learning system mimicking student's ch...,Journal of Physics: Conference Series,© Published under licence by IOP Publishing Lt...,96,0
11564,11968,Machine Learning-Driven and Smartphone-Based F...,ACS Omega,© 2021 The Authors. Published by American Chem...,103,0


# Node

In [38]:
df_count = df_ref.groupby("title_ref").count()

In [52]:
df_ref["title_ref"] = df_ref["title_ref"].str.lower()
df_ref_new = df_ref[df_ref["title_ref"].isin(df_count[df_count["title"] > 1].index)]
df_ref_new.drop(columns=["sourcetitle"],inplace=True)

C:\Users\win12\AppData\Local\Temp\ipykernel_14192\2543759778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref_new.drop(columns=["sourcetitle"],inplace=True)


In [71]:
df_merge = df_ref_new.merge(df_ref_new[["title","title_ref"]],on=["title_ref"],how="left",suffixes=('','_2'))
df_merge = df_merge[df_merge["title"] != df_merge["title_2"]]
df_merge_group = df_merge.groupby("title").count().sort_values("title_ref",ascending=False)

In [74]:
df_merge_group[df_merge_group["title_2"] > 5]

,year,title_ref,title_2
title,,,
Search for low mass vector resonances decaying into quark-antiquark pairs in proton-proton collisions at √s=13 TeV,8421,8421,8421
Search for supersymmetry in proton-proton collisions at √s = 13 TeV in events with high-momentum Z bosons and missing transverse momentum,8192,8192,8192
Inclusive search for highly boosted Higgs bosons decaying to bottom quark-antiquark pairs in proton-proton collisions at √s = 13 TeV,7958,7958,7958
Search for supersymmetry in final states with two oppositely charged same-flavor leptons and missing transverse momentum in proton-proton collisions at √s = 13 TeV,7807,7807,7807
"Search for direct top squark pair production in events with one lepton, jets, and missing transverse momentum at 13 TeV with the CMS experiment",7668,7668,7668
...,...,...,...
Exploring equity in educational policies and interventions in primary and secondary education in the context of public health emergencies: A systematic literature review,6,6,6
Effect of fly ash in Southeast Asia on the properties of mortar,6,6,6
Carbon nanohorns with surface functionalized by plasma treatment and their applications in drug delivery systems,6,6,6


In [ ]:
754045 

In [86]:
group_fre = df_merge.groupby(["title","title_2"]).count().sort_values("title_ref",ascending=False).reset_index()
group_fre.columns = ["title","title_2","count","count_2"]
group_fre = group_fre.drop(columns=["count_2"])

In [89]:
df_dropdup = df_merge.drop_duplicates(subset=["title","title_2"])
df_dropdup = df_dropdup.merge(group_fre,on=["title","title_2"],how="left")

In [93]:
df_2sd = df_dropdup.merge(df_dropdup[["title","title_2"]],left_on="title_2",right_on="title",how="left",suffixes=('','_3'))

In [96]:
df_2sd

,title,year,title_ref,title_2,count,title_3,title_2_3
0,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Update on the pathophysiology and management o...
1,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Factors Associated With Response to Biofeedbac...
2,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Diagnostic performance of dynamic MR defecogra...
3,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Rome foundation-asian working team report: Asi...
4,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Barostat or syringe-assisted sensory biofeedba...
...,...,...,...,...,...,...,...
174210556,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,Migration and sustainable development in Thailand
174210557,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,Do Remittances Increase Household Investment i...
174210558,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,Effects of remittances on household poverty an...
174210559,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,The mediating role of pharmacy engagement on t...


In [ ]:
174210561 

In [95]:
df_2sd[df_2sd["title"] != df_2sd["title_3"]]

,title,year,title_ref,title_2,count,title_3,title_2_3
0,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Update on the pathophysiology and management o...
1,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Factors Associated With Response to Biofeedbac...
2,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Diagnostic performance of dynamic MR defecogra...
3,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Rome foundation-asian working team report: Asi...
4,Public health and international epidemiology f...,2016,"predicting the future - big data, machine lear...",AI-Assisted Diagnosis of Dyssynergic Defecatio...,1.0,AI-Assisted Diagnosis of Dyssynergic Defecatio...,Barostat or syringe-assisted sensory biofeedba...
...,...,...,...,...,...,...,...
174210556,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,Migration and sustainable development in Thailand
174210557,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,Do Remittances Increase Household Investment i...
174210558,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,Effects of remittances on household poverty an...
174210559,Effects of remittances on household poverty an...,2017,economic effects of migration on the left-behi...,Rethinking free skilled labour migration polic...,1.0,Rethinking free skilled labour migration polic...,The mediating role of pharmacy engagement on t...
